In [1]:
import numpy as np
import os
import glob

import librosa
from librosa import load
from librosa import cqt
from librosa.display import specshow

from tqdm import tqdm

In [2]:
hop_size= 512
q= 24

In [3]:
from mirdata import tinysol

dataset = tinysol.load()

In [4]:
features_dict= {}

for key, value in tqdm(list(dataset.items()), disable=False):

    # Read audio files
    waveform, sample_rate= load(value.audio_path, sr=None)

    # Compute CQTs
    cqt_complex= cqt(y=waveform,
                    sr=sample_rate,
                    hop_length=hop_size,
                    bins_per_octave=q,
                    n_bins=q*7,
                    sparsity=1e-6,
                    )
    scalogram= np.abs(cqt_complex)**2

    # Find frame with maximum RMS value
    rms= librosa.feature.rms(y=waveform, hop_length=hop_size)
    rms_argmax= np.argmax(rms)
    feature= scalogram[:,rms_argmax]

    # Stack in dict
    features_dict[key]= feature

  3%|▎         | 73/2913 [00:13<08:23,  5.64it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=456
  n_fft, y.shape[-1]
  3%|▎         | 74/2913 [00:13<07:44,  6.12it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=504
  n_fft, y.shape[-1]
  3%|▎         | 75/2913 [00:13<07:15,  6.52it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=454
  n_fft, y.shape[-1]
  3%|▎         | 76/2913 [00:13<06:49,  6.92it/s]/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=512 is too small for input signal of length=497
  n_fft, y.shape[-1]
  3%|▎         | 78/

In [5]:
import h5py

with h5py.File("TinySOL.h5", "w") as f:
    for key in features_dict.keys():
        f[key] = features_dict[key]

In [6]:
with h5py.File("TinySOL.h5", "r") as f:
    my_dict = {key:f[key][()] for key in f.keys()}

In [7]:
[key for i,key in enumerate(my_dict.keys()) if i in range(500,510)]

['Acc-ord-E3-mf-alt5-N',
 'Acc-ord-E3-pp-N-N',
 'Acc-ord-E3-pp-alt1-N',
 'Acc-ord-E3-pp-alt2-N',
 'Acc-ord-E4-ff-N-N',
 'Acc-ord-E4-ff-alt1-N',
 'Acc-ord-E4-ff-alt2-N',
 'Acc-ord-E4-mf-N-N',
 'Acc-ord-E4-mf-alt1-N',
 'Acc-ord-E4-mf-alt2-N']